In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-01-17 14:48:06--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-01-17 14:48:06 (20.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


So this is our vocabulary or the possible characters thet the model can see.

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


We want a strategy to tokenize the input text.
Toeknize ==> tokenize they mean convert the raw text as a string to some sequence of integers.

here we are going to be building a character level language model so we're simply going to be translating individual characters into integers

 We are building an ENCODER and DECODER.
 encoder: take a string, output a list of integers

 decoder: take a list of integers, output a string

 Encoding chars to Integer: Forward Mapping
 Decoding Integer back to chars: Reverse Mapping

 We have a lookup table now, that stores the char --> Integer encoding.

In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


encode(text):  converts the input text (a string) into a list of integers. Each token represents a unique  character

In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

Data Preparation:
The input text is encoded into numerical format (e.g., tokenized), converted into a PyTorch tensor, and divided into training and validation datasets.
Training and Validation Split:
train_data: First 90% of the data, used for model training.    val_data: Remaining 10%, used to evaluate the model's performance on unseen data.

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

Training Example Creation:
A sequence of tokens (train_data) is split into overlapping chunks of size block_size. Each chunk is used to form pairs of context (input tokens) and target (next token to predict).
The code prepares the data for a model to learn sequential dependencies in text, such as predicting the next token in a sequence given the previous tokens.

    Block Size:
        Defines the length of the context the model uses to predict the next token.
    x and y:
        x: Context/input tokens.
        y: Target tokens (next token corresponding to each context).

For example:

    If train_data is [1, 2, 3, 4, 5, 6, 7, 8, 9] and block_size = 8:
        x = [1, 2, 3, 4, 5, 6, 7, 8]
        y = [2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

    Goal: Simulate how the model processes sequences during training.
    Process:
        For each token position t in block_size:
            context: Includes all tokens up to position t (x[:t+1]).
            target: The next token the model should predict (y[t]).



Illustration of Iteration

For block_size = 4 and x = [1, 2, 3, 4], y = [2, 3, 4, 5]:

    Iteration 1:
        context = [1], target = 2
        "When input is [1], the target: 2"
    Iteration 2:
        context = [1, 2], target = 3
        "When input is [1, 2], the target: 3"
    Iteration 3:
        context = [1, 2, 3], target = 4
        "When input is [1, 2, 3], the target: 4"
    Iteration 4:
        context = [1, 2, 3, 4], target = 5
        "When input is [1, 2, 3, 4], the target: 5"



In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


the block concept here can be likened to a sliding window of size block_size (8 in this case). It moves through the data to extract overlapping sequences of inputs and targets for training.
If the train data has 10 integers and block_size = 8, you need to determine how many valid sliding windows of size 8 can fit into the data.

    Each block of size 8 must have 8 consecutive integers.
    The first block starts at index 0, the second at index 1, and so on, until the window reaches the last possible position where block_size elements fit.

Formula for the Number of Blocks

The formula to calculate the number of valid blocks is:
Number of blocks=Length of train data−block size+1
Number of blocks=Length of train data−block size+1
Calculation for Train Data of Size 10
Number of blocks=10−8+1=3
Number of blocks=10−8+1=3
Example with Train Data of Size 10

Let train_data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] and block_size = 8. The blocks are:

    Block 1: [1, 2, 3, 4, 5, 6, 7, 8]
    Block 2: [2, 3, 4, 5, 6, 7, 8, 9]
    Block 3: [3, 4, 5, 6, 7, 8, 9, 10]

Thus, there are 3 blocks of size 8 in this case.


In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [13]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
